# Pendahuluan

Dalam kode ini, kita akan melakukan pengolahan data untuk melihat pengaruh 10 parameter terhadap kesuksesan penerapan knowledge management di Bank X

# Persiapan

Sebelum menjalankan, buat folder `bank` di Google Drive, kemudian upload data `bank.csv` yang berisi hasil wawancara berupa perbandingan antar parameter. Import dulu semua library yang dibutuhkan

In [ ]:
!pip install ahpy

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Sep 10 15:36:58 2024
"""

import pandas as pd
import numpy as np
import ahpy as ahpy

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification


# Load file

Load file `bank.csv` lalu conversikan ke numpy array

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/My Drive/bank/bank.csv')
np_bank = df.to_numpy()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Initialize parameters

In [ ]:
criteria = ['HC',
            'FC',
            'GCG',
            'RM',
            'BP',
            'IDN',
            'GC',
            'IT',
            'DBM',
            'OA']

success_criteria = [
        'success',
        'failed'
    ]

weights = np.zeros((np_bank.shape[0], 2*len(criteria)))

Run AHP

In [ ]:
cr = []
for data_index in range(0, np_bank.shape[0]):
    print(f'Data number {data_index}')

    data_sample = np_bank[data_index][:46]

    np_weight = np.zeros((len(criteria), len(criteria)))

    idx = 1
    leng = len(criteria) - 1

    for i in range(0, len(criteria)):
        if i < len(criteria):
            data = data_sample[idx: idx + leng]
            np_weight[i, (i+1):] = data
            idx = idx + leng
            leng = leng - 1
        for j in range(0, len(criteria)):
            if i == j:
                np_weight[i][j] = 1
            if i > j:
                np_weight[i][j] = 1 / np_weight[j][i]

    # Convert the numpy matrix into a dictionary of comparisons for ahpy
    comparison_dict = {}
    for i in range(len(criteria)):
        for j in range(i+1, len(criteria)):
            comparison_dict[(criteria[i], criteria[j])] = np_weight[i][j]

    # Initialize the ahpy Compare object
    ahp_compare = ahpy.Compare(name='AHP_Model', comparisons=comparison_dict, precision=3)

    # Print the priority vector (weights)

    target_weights = ahp_compare.target_weights
    print("Criteria Weights:")
    print(target_weights)

    # Print the consistency ratio
    print("Consistency Ratio:")
    print(ahp_compare.consistency_ratio)
    cr.append(ahp_compare.consistency_ratio)

    temp = np.zeros((1, len(criteria)))

    for j in range(0, len(criteria)):
        crit = target_weights[criteria[j]]
        temp[0, j] = crit

    # put criteria matrix as input
    weights[data_index][:10] = temp

    data_sample_success = np_bank[data_index][46:56]

    param_matrix = np.zeros((10, 2, 2))

    success = []
    data_idx = 0
    for data in data_sample_success:
        for i in range(0, 2):
            for j in range(0, 2):
                if i == j:
                    param_matrix[data_idx][i][j] = 1
        param_matrix[data_idx][0][1] = data_sample_success[data_idx]
        param_matrix[data_idx][1][0] = 1 / float(data_sample_success[data_idx])

        # Convert the numpy matrix into a dictionary of comparisons for ahpy
        comparison_dict = {}
        for i in range(0, 2):
            for j in range(0, 2):
                comparison_dict[(success_criteria[i], success_criteria[j])] = param_matrix[data_idx][i][j]

        # Initialize the ahpy Compare object
        ahp_compare = ahpy.Compare(name='AHP_Model', comparisons=comparison_dict, precision=3)

        # Print the priority vector (weights)
        print(f'Parameter: {criteria[data_idx]}')

        target_weights_success = ahp_compare.target_weights
        print("Criteria Weights:")
        print(target_weights_success)
        success.append(target_weights_success['success'])
        print()
        data_idx += 1

    # put success rate as output
    weights[data_index][10:20] = success

Data number 0
Criteria Weights:
{'HC': 0.194, 'FC': 0.173, 'RM': 0.153, 'OA': 0.114, 'IT': 0.112, 'GCG': 0.085, 'IDN': 0.05, 'BP': 0.049, 'DBM': 0.041, 'GC': 0.028}
Consistency Ratio:
0.055
Parameter: HC
Criteria Weights:
{'success': 0.889, 'failed': 0.111}

Parameter: FC
Criteria Weights:
{'success': 0.875, 'failed': 0.125}

Parameter: GCG
Criteria Weights:
{'success': 0.75, 'failed': 0.25}

Parameter: RM
Criteria Weights:
{'success': 0.857, 'failed': 0.143}

Parameter: BP
Criteria Weights:
{'success': 0.5, 'failed': 0.5}

Parameter: IDN
Criteria Weights:
{'success': 0.667, 'failed': 0.333}

Parameter: GC
Criteria Weights:
{'failed': 0.875, 'success': 0.125}

Parameter: IT
Criteria Weights:
{'success': 0.8, 'failed': 0.2}

Parameter: DBM
Criteria Weights:
{'failed': 0.833, 'success': 0.167}

Parameter: OA
Criteria Weights:
{'success': 0.833, 'failed': 0.167}

Data number 1
Criteria Weights:
{'HC': 0.212, 'FC': 0.168, 'RM': 0.126, 'OA': 0.126, 'IT': 0.122, 'GCG': 0.084, 'BP': 0.048, 'I

Run ANN

In [ ]:
weight_df = pd.DataFrame(weights)
weight_df['cr'] = cr
weight_df.to_csv('/content/drive/My Drive/bank/weight.csv', index=False)

Masukkan input yang akan diprediksi success ratenya

In [ ]:

X_train, X_test, Y_train, Y_test = train_test_split(weights[:, :10], weights[:, 10:], test_size=0.3, random_state=42)

np_ytrain = np.array(Y_train)
np_ytest = np.array(Y_test)

squared_ytrain = np_ytrain ** 3
squared_ytest = np_ytest ** 3

# Initialize the scaler
scaler = MinMaxScaler()

# Fit and transform the training data
X_train = scaler.fit_transform(X_train)

# Only transform the test data
X_test = scaler.transform(X_test)

model = Sequential()
model.add(Dense(100, input_dim=10, activation='softmax')) # relu
model.add(Dropout(0.2))  # Dropout layer to reduce overfitting
model.add(Dense(10, activation='sigmoid'))

# Compile the model (categorical crossentropy for multi-class classification)
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=500, batch_size=10, validation_data=(X_test, Y_test))

Epoch 1/500
5/5 [==============================] - 1s 39ms/step - loss: 0.6904 - accuracy: 0.0204 - val_loss: 0.6886 - val_accuracy: 0.0000e+00
Epoch 2/500
5/5 [==============================] - 0s 11ms/step - loss: 0.6872 - accuracy: 0.0000e+00 - val_loss: 0.6854 - val_accuracy: 0.0000e+00
Epoch 3/500
5/5 [==============================] - 0s 11ms/step - loss: 0.6842 - accuracy: 0.0000e+00 - val_loss: 0.6823 - val_accuracy: 0.0000e+00
Epoch 4/500
5/5 [==============================] - 0s 11ms/step - loss: 0.6811 - accuracy: 0.0408 - val_loss: 0.6792 - val_accuracy: 0.0000e+00
Epoch 5/500
5/5 [==============================] - 0s 11ms/step - loss: 0.6782 - accuracy: 0.0408 - val_loss: 0.6762 - val_accuracy: 0.0000e+00
Epoch 6/500
5/5 [==============================] - 0s 15ms/step - loss: 0.6749 - accuracy: 0.0408 - val_loss: 0.6732 - val_accuracy: 0.0000e+00
Epoch 7/500
5/5 [==============================] - 0s 10ms/step - loss: 0.6722 - accuracy: 0.1020 - val_loss: 0.6703 - val_accur

In [ ]:
HC = 0.212
FC = 0.168
GCG = 0.084
RM = 0.126
BP = 0.048
IDN = 0.044
GC = 0.029
IT = 0.122
DBM = 0.042
OA = 0.126

input_data = np.array([HC, FC, GCG, RM, BP, IDN, GC, IT, DBM, OA])

In [ ]:
sample_input = weights[1, :10]
sample_output = weights[1, 10:]

In [ ]:
prediction_result = model.predict(input_data.reshape(1, -1))

1/1 [==============================] - 0s 24ms/step


In [ ]:
print([round(num, 2) for num in prediction_result[0]])
print(sample_output)

[0.82, 0.81, 0.72, 0.78, 0.45, 0.63, 0.2, 0.74, 0.23, 0.77]
[0.9   0.875 0.75  0.833 0.333 0.25  0.125 0.75  0.167 0.857]
